# Goals

The Goal for is this to provide a proper Visualization of COVID-19

# Imports

In [44]:
import pandas as pd
from datetime import datetime, date
import plotly.offline as py
import plotly.graph_objs as go
import plotly.express as px
import numpy as np

In [45]:
data = pd.read_csv("covid_19_clean_complete.csv")

In [46]:
data.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Thailand,15.0000,101.0000,1/22/20,2,0,0
1,NaN,Japan,36.0000,138.0000,1/22/20,2,0,0
2,NaN,Singapore,1.2833,103.8333,1/22/20,0,0,0
3,NaN,Nepal,28.1667,84.2500,1/22/20,0,0,0
4,NaN,Malaysia,2.5000,112.5000,1/22/20,0,0,0


### Format Date/Get Unique Dates

In [47]:
formatter_string = "%m/%d/%y"
data['date'] = pd.to_datetime(data.Date, format=formatter_string)

In [48]:
#Get unique Dates and Max Date
date_data = data.date.unique()
max_data_date = max(date_data)

In [49]:
print("Max Time RN {0}".format(max_data_date))

Max Time RN 2020-03-17T00:00:00.000000000


In [50]:
d = pd.to_datetime(str(max_data_date)).strftime('%Y-%m-%d')

title = 'COVID-19'
off_title = title + ' as of ' + d 

In [51]:
off_title

'COVID-19 as of 2020-03-17'

# Plot Cases Over Time

In [52]:
py.offline.init_notebook_mode(connected = True)

In [53]:
cases_df = data[['date','Confirmed','Deaths', 'Recovered']].groupby('date').sum()

graph_confirm = go.Scatter(x=cases_df.index, y = cases_df.Confirmed, mode="markers+lines", name="Confirmed")

graph_deaths = go.Scatter(x=cases_df.index, y = cases_df.Deaths, mode="markers+lines", name="Deaths")

recover = go.Scatter(x=cases_df.index, y = cases_df.Recovered, mode="markers+lines", name="Recovered")




grapher = [graph_confirm, graph_deaths, recover]

py.iplot({
    "data": grapher,
    "layout": go.Layout(title=off_title)
})

# Get Most Current Data

In [54]:
df = data.loc[data.date==max_data_date]


In [55]:
by_country = df[['Country/Region', 'Confirmed','Deaths', 'Recovered']].groupby(['Country/Region']).sum()

by_country.sort_values(by='Confirmed', ascending = False, inplace = True)

by_country['Active'] = by_country['Confirmed'] - by_country['Deaths'] - by_country['Recovered']

# Pie Chart Visualization

In [57]:
countries = by_country.index.to_list()
ids = countries
labels = countries
parents = [title] * len(countries)
values = by_country.Confirmed.to_list()

classifications = by_country.columns.drop('Confirmed')

for cty in by_country.index:
    for c in classifications:
        ids = ids + [cty + '_' + c]
        parents = parents + [cty]
        labels = labels + [c]
        values = values + [by_country.loc[cty][c]]

trace = go.Sunburst(
    ids=ids,
    labels=labels,
    parents=parents,
    values=values,
    branchvalues="total",
    outsidetextfont={"size": 25, "color": "#377eb8"},
    marker={"line": {"width": 2}}
)

layout = go.Layout(
    title = off_title + "<br>(click on country)",
    margin = go.layout.Margin(t=100, l=0, r=0, b=0),
    sunburstcolorway=["#636efa","#ef553b","#00cc96"]
)

fig = go.Figure([trace], layout)

py.iplot(fig)